## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)

In [4]:
# Determine the number of unique values in each column
unique_values = application_df.nunique()

# Display the result
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Display value counts for APPLICATION_TYPE
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [6]:
# Choose a cutoff value
cutoff_value = 500

# Identify application types to be replaced based on the cutoff
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Create the list of application types to be replaced
application_types_to_replace = types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check the value counts to ensure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [7]:
# Display value counts for CLASSIFICATION
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [8]:
# Display value counts for CLASSIFICATION where count is greater than 1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts_greater_than_1 = classification_counts[classification_counts > 1]
print(classification_counts_greater_than_1)


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [9]:
# Choose a cutoff value
cutoff_value_classification = 1000  # You can adjust this value based on your analysis

# Identify classifications to be replaced based on the cutoff
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < cutoff_value_classification].index.tolist()

# Create the list of classifications to be replaced
classifications_to_replace = classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check the value counts to ensure binning was successful
print(application_df['CLASSIFICATION'].value_counts())


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [10]:
# Use pd.get_dummies to convert categorical data to numeric
application_df = pd.get_dummies(application_df, columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"])

# Display the updated dataframe
print(application_df.head())


   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                       0   
1       1   108590              1                       0   
2       1     5000              0                       0   
3       1     6692              1                       0   
4       1   142590              1                       0   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                     1                     0                    0   
1                     0                     0                    1   
2                     0                     0                    0   
3                     0                     0                    1   
4                     0                     0                    1   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                    0                    0                    0  ...   
1                    0                    0                    0  

In [12]:
# Assuming 'IS_SUCCESSFUL' is your target variable
X = application_df.drop(columns=["IS_SUCCESSFUL"])
y = application_df["IS_SUCCESSFUL"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verify the shapes of your training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (25724, 43)
X_test shape: (8575, 43)
y_train shape: (25724,)
y_test shape: (8575,)


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=120, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer (you can adjust the number of units as needed)
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 120)               5280      
                                                                 
 dense_7 (Dense)             (None, 50)                6050      
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11381 (44.46 KB)
Trainable params: 11381 (44.46 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [19]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=300, batch_size=32, validation_data=(X_test_scaled, y_test))


Epoch 1/300
804/804 [==============================] - 4s 3ms/step - loss: 0.5672 - accuracy: 0.7264 - val_loss: 0.5651 - val_accuracy: 0.7213
Epoch 2/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5551 - accuracy: 0.7308 - val_loss: 0.5635 - val_accuracy: 0.7214
Epoch 3/300
804/804 [==============================] - 2s 3ms/step - loss: 0.5521 - accuracy: 0.7316 - val_loss: 0.5581 - val_accuracy: 0.7240
Epoch 4/300
804/804 [==============================] - 3s 4ms/step - loss: 0.5495 - accuracy: 0.7337 - val_loss: 0.5621 - val_accuracy: 0.7242
Epoch 5/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7332 - val_loss: 0.5538 - val_accuracy: 0.7262
Epoch 6/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7339 - val_loss: 0.5571 - val_accuracy: 0.7255
Epoch 7/300
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7355 - val_loss: 0.5556 - val_accuracy: 0.7256

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5613 - accuracy: 0.7261 - 383ms/epoch - 1ms/step
Loss: 0.5613272786140442, Accuracy: 0.726064145565033


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")